# Build FakeHuayi backend

## Basic Use

Required files:

- ./huayi_providers/fake_huayi/
    - \_\_init\_\_.py
    - fake_huayi.py
    - props_huayi.json
    - conf_huayi.json
    - defs_huayi.json (if Pulse Backend is applied, TODO)
- ./
    - qubits_data.csv
    - gates_data.csv

Import FakeHuayi backend

```
from huayi_providers.fake_huayi import *
FackHuayi()    # for V1 backend
FackHuayiV2()  # for V2 backend
```

Generate noise model
```
from qiskit_aer.noise.noise_model import NoiseModel
noise_Huayi = NoiseModel.from_backend(FakeHuayi())
```

Create .json files that required to build the backend

```
from huayi_providers.backend_build import build_from_file
c = build_from_file(backend_name="huayi",
                    backend_version="x.x.x",
                    qubits_data="qubits_data.csv",
                    gates_data="gates_data.csv")
```

## File Structure

#### Expriment data

**qubits_data.csv** contains the information of qubits, including
- T1 time (ms)
- T2 time (ms)
- frequency (MHz)
- readout error rate
- Probability of finding 0 when prepared in 1
- Probability of finding 1 when prepared in 0
- readout length (us)

All information should be accompanied with the measurement date and time.

**gates_data.csv** contains the information of gates, including
- qubits
- gate type
- error rate
- length
- gate name (optional)

The gate error and length are measured from experiment, and should be accompanied with the measurement date and time.

#### Dictionaries of the backend properties and configurations

**props_huayi.json**
```
{'backend_name': 'fakehuayi',
 'backend_version='0.0.1',
 'last_update_date': now_time(),
 'qubits': [q1, q2, ...],
 'gates': [g1, g2, ...],
 'general': []}
```
 qubit info (q):
```
 [{'date': ['T1_date'], 'name': 'T1', 'unit': 'ms', 'value': ['T1']},
  {'date': ['T2_date'], 'name': 'T2', 'unit': 'ms', 'value': ['T2']},
  {'date': ['frequency_date'], 'name': 'frequency', 'unit': 'MHz', 'value': ['frequency']},
  {'date': ['readout_error_date'], 'name': 'readout_error', 'unit': '', 'value': ['readout_error']},
  {'date': ['prob_meas0_prep1_date'], 'name': 'prob_meas0_prep1', 'unit': '', 'value': ['prob_meas0_prep1']},
  {'date': ['prob_meas1_prep0_date'], 'name': 'prob_meas1_prep0', 'unit': '', 'value': ['prob_meas1_prep0']},
  {'date': ['readout_length_date'], 'name': 'readout_length', 'unit': 'us', 'value': ['readout_length']}]
```
gate info (g):
```
{'qubits': ['qubits'],
 'gate': ['gate'],
 'parameters': [{'date': ['error_date'],
                 'name': 'gate_error',
                 'unit': '',
                 'value': ['gate_error']},
                {'date': ['length_date'],
                 'name': 'gate_length',
                 'unit': 'ms',
                 'value': ['gate_length']}],
 'name': ['name']}
```

**conf_huayi.json**

#### Dictionary of the Noise Model

The keys in NoiseModel are
- basis gates ({'id', 'x', 'sx', 'rz', 'cx'})
- noise instructions ({'id', 'x', 'sx', 'cx', 'measure', 'reset', ''})
- noise qubits ({0, ... , 26})
- default quantum errors (set None by default)
- default readout errors (set None by default)
- local quantum errors
- local readout errors
- custom noise passes (set None by default)

## Build a backend from csv files

In [42]:
# Generate props_huayi.json from .csv experimental data
from huayi_providers.backend_build import build_from_file

"""
'gates_radius' defines a coupling_map with the distance between two-qubit gates smaller (or equal) to gates_radius
But suggest not to use 'gates_radius', but define the 'coupling_map' in transpile()

use the following map for finite-range coupling:

       coupling_map = [[i,j] for i in range(n_qubits)
                             for j in list(range(max(0,i-gates_radius),i))+
                                      list(range(i+1,min(i+1+gates_radius,n_qubits)))]
"""
c = build_from_file(backend_name='huayi8',
       backend_version='0.0.1',
       qubits_data='qubits_data_8.csv', 
       gates_data='gates_data_8.csv',)


Successfully created props_huayi8.json
Successfully created conf_huayi8.json
New backends created, please import the backends with:
from huayi_providers.fake_huayi8 import FakeHuayi8, FakeHuayi8V2


## Check if the new backend works

In [63]:
from huayi_providers.fake_huayi8 import FakeHuayi8, FakeHuayi8V2
from qiskit.circuit.library import EfficientSU2
from qiskit import transpile
from qiskit import QuantumCircuit

n_qubits = 4
ansatz = EfficientSU2(n_qubits, 
                      su2_gates=['ry'], 
                      entanglement="linear", 
                      reps=1,
                      flatten=True)
print("Efficient SU2 ansatz")
print(ansatz.draw(fold=140, idle_wires=False))


coupling_map = [[i,j] for i in range(8)
                      for j in list(range(max(0,i-3),i))+
                               list(range(i+1,min(i+1+3,8)))]
c_huayi = transpile(ansatz, backend=FakeHuayi8(), optimization_level=3)
print("Transpileed with Huayi backend, depth = {}".format(c_huayi.depth()))
print(c_huayi.draw(fold=140, idle_wires=False))


Efficient SU2 ansatz
     ┌──────────┐     ┌──────────┐                        
q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[4]) ├────────────────────────
     ├──────────┤┌─┴─┐└──────────┘┌──────────┐            
q_1: ┤ Ry(θ[1]) ├┤ X ├─────■──────┤ Ry(θ[5]) ├────────────
     ├──────────┤└───┘   ┌─┴─┐    └──────────┘┌──────────┐
q_2: ┤ Ry(θ[2]) ├────────┤ X ├─────────■──────┤ Ry(θ[6]) ├
     ├──────────┤        └───┘       ┌─┴─┐    ├──────────┤
q_3: ┤ Ry(θ[3]) ├────────────────────┤ X ├────┤ Ry(θ[7]) ├
     └──────────┘                    └───┘    └──────────┘
Transpileed with Huayi backend, depth = 13
global phase: π
         ┌──────────┐┌─────────┐┌───────┐   ┌─────────┐ ┌───────┐   ┌──────────┐                                             
q_1 -> 2 ┤ Ry(θ[1]) ├┤ Ry(π/2) ├┤ Rx(π) ├─■─┤ Ry(π/2) ├─┤ Rx(π) ├─■─┤ Ry(θ[5]) ├─────────────────────────────────────────────
         ├──────────┤├─────────┤├───────┤ │ └─────────┘ └───────┘ │ └──────────┘            ┌─────────┐ ┌───────┐┌──────────┐
q_3 -> 3 ┤ 

I have manually forced the CZ gate error very large (~0.8) except for the couplings between [1,3], [1,5] and [3,7]
The transpiled circuit indeed only connects the low-error qubits

## Create random properties of qubits and gates

In [41]:
import pandas as pd
import random
from huayi_providers.backend_build import now_time

n_qubits = 8

qubits_info = pd.DataFrame(columns=['T1','T1_date', 
                                    'T2','T2_date', 
                                    'frequency', 'frequency_date', 
                                    'readout_error', 'readout_error_date', 
                                    'prob_meas0_prep1','prob_meas0_prep1_date', 
                                    'prob_meas1_prep0', 'prob_meas1_prep0_date', 
                                    'readout_length', 'readout_length_date'])
for i in range(n_qubits):
    qubits_info.loc[i] = [abs(random.gauss(50e3, 1e3)), now_time(),
                         abs(random.gauss(1000, 50)), now_time(),
                         abs(random.gauss(1, 0.05)), now_time(),
                         abs(random.gauss(0, 0.05e-2)), now_time(),
                         abs(random.gauss(0, 0.03e-2)), now_time(),
                         abs(random.gauss(0, 0.03e-2)), now_time(),
                         abs(random.gauss(0.1, 0.01)), now_time(),]
with open('qubits_data_{}.csv'.format(n_qubits), 'w') as f:
    qubits_info.to_csv(f)


gates_info = pd.DataFrame(columns=['qubits',
                                   'gate',
                                   'gate_error', 'error_date',
                                   'gate_length', 'length_date',
                                   'name'])
iter = -1
for gate in ['id', 'rx' ,'ry']:
    for j in range(n_qubits):
        iter += 1
        gates_info.loc[iter] = ['['+str(j)+']',
                                gate,
                                abs(random.gauss(0, 0.0005)), now_time(),
                                random.gauss(150, 10), now_time(),
                                gate+str(j)]
for gate in ['cz']:
    for i in range(n_qubits):
        for j in list(range(i))+list(range(i+1,n_qubits)):
            iter += 1
            gates_info.loc[iter] = ['['+str(i)+','+str(j)+']',
                                    gate,
                                    abs(random.gauss(0, 0.01)), now_time(),
                                    abs(random.gauss(150, 10)), now_time(),
                                    gate+str(i)+'_'+str(j)]
    

with open('qubits_data_{}.csv'.format(n_qubits), 'w') as f:
    qubits_info.to_csv(f, index=False, lineterminator='\n')
with open('gates_data_{}.csv'.format(n_qubits), 'w') as f:
    gates_info.to_csv(f, index=False, lineterminator='\n')


display(pd.read_csv('qubits_data_{}.csv'.format(n_qubits)))
display(pd.read_csv('gates_data_{}.csv'.format(n_qubits)))

,T1,T1_date,T2,T2_date,frequency,frequency_date,readout_error,readout_error_date,prob_meas0_prep1,prob_meas0_prep1_date,prob_meas1_prep0,prob_meas1_prep0_date,readout_length,readout_length_date
0,48999.396141,2023-12-21T10:54+08:00,992.927742,2023-12-21T10:54+08:00,1.040723,2023-12-21T10:54+08:00,0.000050,2023-12-21T10:54+08:00,0.000297,2023-12-21T10:54+08:00,0.000079,2023-12-21T10:54+08:00,0.084875,2023-12-21T10:54+08:00
1,50954.660677,2023-12-21T10:54+08:00,960.352023,2023-12-21T10:54+08:00,0.955275,2023-12-21T10:54+08:00,0.000293,2023-12-21T10:54+08:00,0.000026,2023-12-21T10:54+08:00,0.000493,2023-12-21T10:54+08:00,0.104056,2023-12-21T10:54+08:00
2,48929.479609,2023-12-21T10:54+08:00,955.825224,2023-12-21T10:54+08:00,1.034251,2023-12-21T10:54+08:00,0.000431,2023-12-21T10:54+08:00,0.000568,2023-12-21T10:54+08:00,0.000220,2023-12-21T10:54+08:00,0.107140,2023-12-21T10:54+08:00
3,50059.712192,2023-12-21T10:54+08:00,925.838895,2023-12-21T10:54+08:00,1.028481,2023-12-21T10:54+08:00,0.000109,2023-12-21T10:54+08:00,0.000358,2023-12-21T10:54+08:00,0.000378,2023-12-21T10:54+08:00,0.094982,2023-12-21T10:54+08:00
4,49094.115984,2023-12-21T10:54+08:00,985.590662,2023-12-21T10:54+08:00,1.069440,2023-12-21T10:54+08:00,0.000159,2023-12-21T10:54+08:00,0.000095,2023-12-21T10:54+08:00,0.000079,2023-12-21T10:54+08:00,0.112753,2023-12-21T10:54+08:00
5,49059.498885,2023-12-21T10:54+08:00,1000.782212,2023-12-21T10:54+08:00,1.056639,2023-12-21T10:54+08:00,0.000508,2023-12-21T10:54+08:00,0.000423,2023-12-21T10:54+08:00,0.000455,2023-12-21T10:54+08:00,0.111560,2023-12-21T10:54+08:00
6,50280.995645,2023-12-21T10:54+08:00,933.069773,2023-12-21T10:54+08:00,0.980321,2023-12-21T10:54+08:00,0.000308,2023-12-21T10:54+08:00,0.000293,2023-12-21T10:54+08:00,0.000289,2023-12-21T10:54+08:00,0.092623,2023-12-21T10:54+08:00
7,51741.660643,2023-12-21T10:54+08:00,957.031828,2023-12-21T10:54+08:00,1.038080,2023-12-21T10:54+08:00,0.000090,2023-12-21T10:54+08:00,0.000079,2023-12-21T10:54+08:00,0.000035,2023-12-21T10:54+08:00,0.099236,2023-12-21T10:54+08:00


,qubits,gate,gate_error,error_date,gate_length,length_date,name
0,[0],id,0.000099,2023-12-21T10:54+08:00,162.789619,2023-12-21T10:54+08:00,id0
1,[1],id,0.000453,2023-12-21T10:54+08:00,142.018224,2023-12-21T10:54+08:00,id1
2,[2],id,0.000289,2023-12-21T10:54+08:00,129.663965,2023-12-21T10:54+08:00,id2
3,[3],id,0.000285,2023-12-21T10:54+08:00,148.381416,2023-12-21T10:54+08:00,id3
4,[4],id,0.000078,2023-12-21T10:54+08:00,169.625269,2023-12-21T10:54+08:00,id4
...,...,...,...,...,...,...,...
75,"[7,2]",cz,0.005851,2023-12-21T10:54+08:00,152.235268,2023-12-21T10:54+08:00,cz7_2
76,"[7,3]",cz,0.002574,2023-12-21T10:54+08:00,156.520076,2023-12-21T10:54+08:00,cz7_3
77,"[7,4]",cz,0.010868,2023-12-21T10:54+08:00,150.362785,2023-12-21T10:54+08:00,cz7_4
78,"[7,5]",cz,0.009498,2023-12-21T10:54+08:00,150.672060,2023-12-21T10:54+08:00,cz7_5
